In [2]:
#This notebook is used to fetch or load raw data and save it in the artifacts/data_ingestion directory.
#It also unzips the data if it is in a zip format.
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\hp\\Desktop\\Projects\\Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from text_summarizer.constant import *
from text_summarizer.utils.common import read_yaml, create_directories


In [7]:
print("Using config file:", CONFIG_FILE_PATH)


Using config file: config\config.yaml


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        
        return data_ingestion_config

In [9]:
import os
import urllib.request as request
import zipfile
from text_summarizer.logging import logger
from text_summarizer.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2025-08-05 10:57:29,862  - INFO - common - yaml file: config\config.yaml loaded successfully
2025-08-05 10:57:29,864  - INFO - common - yaml file: params.yaml loaded successfully
2025-08-05 10:57:29,865  - INFO - common - created directory at: artifacts
2025-08-05 10:57:29,867  - INFO - common - created directory at: artifacts/data_ingestion
2025-08-05 10:57:29,868  - INFO - 3078253932 - File already exists of size: ~ 7718 KB
